1. Some values in the the FlightNumber column are missing. These numbers are
meant to increase by 10 with each row so 10055 and 10075 need to be put in
place. Fill in these missing numbers and make the column an integer column

In [119]:
import pandas as pd
import numpy as np

df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN',
'londON_StockhOlm',
'Budapest_PaRis', 'Brussels_londOn'],
'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],
'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],
'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )',
'12. Air France', '"Swiss Air"']})

df.interpolate(inplace=True)
df['FlightNumber']=df['FlightNumber'].astype(dtype='int64')

df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045,"[23, 47]",KLM(!)
1,MAdrid_miLAN,10055,[],<Air France> (12)
2,londON_StockhOlm,10065,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,10075,[13],12. Air France
4,Brussels_londOn,10085,"[67, 32]","""Swiss Air"""


2.The From_To column would be better as two separate columns! Split each
string on the underscore delimiter _ to give a new temporary DataFrame with
the correct values. Assign the correct column names to this temporary
DataFrame.

In [120]:
df_temp=df.copy()

new=df_temp['From_To'].str.split('_',expand=True)

df_temp['From']=new[0]
df_temp['To']=new[1]

df_temp.drop(columns='From_To',inplace=True)
print(df_temp)


   FlightNumber  RecentDelays              Airline      From         To
0         10045      [23, 47]               KLM(!)    LoNDon      paris
1         10055            []    <Air France> (12)    MAdrid      miLAN
2         10065  [24, 43, 87]  (British Airways. )    londON  StockhOlm
3         10075          [13]       12. Air France  Budapest      PaRis
4         10085      [67, 32]          "Swiss Air"  Brussels     londOn


3.Notice how the capitalisation of the city names is all mixed up in this
temporary DataFrame. Standardise the strings so that only the first letter is
uppercase (e.g. "londON" should become "London".)


In [121]:
def Standardise_series(df_temp,ser):
  df_temp[ser]=df_temp[ser].str.lower()
  for i in range(df_temp[ser].size):
    df_temp[ser][i]=df_temp[ser][i][0].upper()+df_temp[ser][i][1:]
  return df_temp

df_temp=Standardise_series(df_temp,'From')
df_temp=Standardise_series(df_temp,'To')

print(df_temp)

   FlightNumber  RecentDelays              Airline      From         To
0         10045      [23, 47]               KLM(!)    London      Paris
1         10055            []    <Air France> (12)    Madrid      Milan
2         10065  [24, 43, 87]  (British Airways. )    London  Stockholm
3         10075          [13]       12. Air France  Budapest      Paris
4         10085      [67, 32]          "Swiss Air"  Brussels     London


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


4. Delete the From_To column from df and attach the temporary DataFrame
from the previous questions.

In [122]:
df.drop(['From_To'],axis=1,inplace=True)
df[['From','To']]=df_temp[['From','To']]
df

,FlightNumber,RecentDelays,Airline,From,To
0,10045,"[23, 47]",KLM(!),London,Paris
1,10055,[],<Air France> (12),Madrid,Milan
2,10065,"[24, 43, 87]",(British Airways. ),London,Stockholm
3,10075,[13],12. Air France,Budapest,Paris
4,10085,"[67, 32]","""Swiss Air""",Brussels,London


5. In the RecentDelays column, the values have been entered into the
DataFrame as a list. We would like each first value in its own column, each 
second value in its own column, and so on. If there isn't an Nth value, the value
should be NaN.


In [123]:
from numpy.random import randn as rn
lsts=[]
for i in range(df['RecentDelays'].size):
  lsts.append((df['RecentDelays'][i]))



l1=['RecentDelays','RecentDelays','RecentDelays']
l2=[1,2,3]

t3=tuple(zip(l1,l2))
ind=pd.MultiIndex.from_tuples(t3)
dp=pd.DataFrame(data=lsts,columns=ind)

lsts=pd.DataFrame(lsts,columns=['Delay_1','Delay_2','Delay_3'])

print('Splitting Delays\n',lsts)
print('\n')
print('Split using multiindex\n',dp)
 

Splitting Delays
    Delay_1  Delay_2  Delay_3
0     23.0     47.0      NaN
1      NaN      NaN      NaN
2     24.0     43.0     87.0
3     13.0      NaN      NaN
4     67.0     32.0      NaN


Split using multiindex
   RecentDelays            
             1     2     3
0         23.0  47.0   NaN
1          NaN   NaN   NaN
2         24.0  43.0  87.0
3         13.0   NaN   NaN
4         67.0  32.0   NaN


5.Expand the Series of lists into a DataFrame named delays, rename the columns
delay_1, delay_2, etc. and replace the unwanted RecentDelays column in df
with delays.

In [124]:
df.drop('RecentDelays',axis=1,inplace=True)
df=pd.concat([df,lsts],axis=1)
df=df[['FlightNumber','Delay_1','Delay_2','Delay_3','Airline','From','To']]
df


,FlightNumber,Delay_1,Delay_2,Delay_3,Airline,From,To
0,10045,23.0,47.0,NaN,KLM(!),London,Paris
1,10055,NaN,NaN,NaN,<Air France> (12),Madrid,Milan
2,10065,24.0,43.0,87.0,(British Airways. ),London,Stockholm
3,10075,13.0,NaN,NaN,12. Air France,Budapest,Paris
4,10085,67.0,32.0,NaN,"""Swiss Air""",Brussels,London
